There is only one file for Chapter 3, as there is really only one kind of visual, and most of the chapter is arguing the recurrences. For this file, the first two blocks involve loading the packages and defining some useful helper functions. For the simulation, make sure to set the `bootstrapping` variable properly and to uncomment the lines of code that correspond to the proper calculation of the recurrence. Also, change the line in the block that creates the visual.

In [ ]:
import numpy as np
import scipy
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from collections import defaultdict
from copy import deepcopy
from matplotlib import pyplot
from tqdm import tqdm

In [ ]:
# Data generating functions
def multivariateNorm(params):
    muVec = np.zeros(params['dim'])
    covMatrix = np.diag(np.ones(params['dim']))

    covMatrix[0,1] = params['rho']
    covMatrix[1,0] = params['rho']

    X = np.random.multivariate_normal(muVec,covMatrix,params['sampleSize'])
    epsilon = np.random.normal(0,0.1,params['sampleSize'])
    y = np.sum(X,axis = 1) + epsilon

    return X,y

def onlyDependent(params):
    thetas = np.random.uniform(low = 0.0, high = 2*np.pi, size = params['sampleSize'])
    x1 = np.sin(thetas)
    x2 = np.cos(thetas)

    otherVars = np.random.uniform(0,1,(params['dim']-2,params['sampleSize']))
    
    epsilon = np.random.normal(0,0.1,params['sampleSize'])

    X = np.concatenate((np.array((x1,x2)).T,otherVars.T), axis = 1)
    y = np.sum(X,axis = 1) + epsilon

    return X,y

def blocks(params):
    blockIdxStart = 2 # Want the first two features to be independent of everyone

    covMatrix = np.zeros((params['dim'],params['dim']))
    muVec = np.zeros(params['dim'])

    while blockIdxStart +params['blockSize'] <= params['dim']:
        covMatrix[blockIdxStart:blockIdxStart+params['blockSize'],blockIdxStart:blockIdxStart+params['blockSize']] = params['rho']
        blockIdxStart += params['blockSize']
    
    for i in range(params['dim']):
        covMatrix[i,i] = 1
    
    epsilon = np.random.normal(0,0.1,params['sampleSize'])

    X = np.random.multivariate_normal(muVec,covMatrix,params['sampleSize'])
    y = np.sum(X,axis = 1) + epsilon

    return X,y

In [ ]:
ts = [0.01,0.05,0.25,0.75,1.0,1.5,2.0]
Ts = [500,1000,1500,2000]
ks = [1,3,5,15]
repetitions = 10
dataGenerators = [multivariateNorm,onlyDependent,blocks]
variables = 10
rho = 0.75
bootstrapping = False # To recreate the analysis for bagged trees, set this to True
dataGenDict = defaultdict(list)

c = 1 - 1/np.e

for gen in dataGenerators:
    overallComputationsDict = defaultdict(list)
    uniqueComputationsDict = defaultdict(list)
    lDict = defaultdict(list)
    recDict = defaultdict(list)
    for T in tqdm(Ts):
        for t in ts:
            for k in ks:
                for _ in range(repetitions):
                    # Generate Data
                    X,y = gen({'dim' : variables, 'sampleSize' : T, 'rho' : rho, 'blockSize' : 3})
                    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.25, random_state=31)
                    
                    # Grow the forest
                    forest = RandomForestRegressor(n_estimators = int(t*T), min_samples_leaf = k, n_jobs = -1,random_state = 31, max_features = 'sqrt',bootstrap = bootstrapping)
                    forest.fit(xTrain, yTrain)

                    trainObsInLeaves = defaultdict(list)
                    trainingIndexList = forest.estimators_samples_ # indices of observations used in the fitting of each tree
                    trees = forest.estimators_ # the decision tree structures

                    for treeIdx,bootIndices in enumerate(trainingIndexList): # For each bootstrap tree...
                        bootData = xTrain[bootIndices,] # Get the data used to fit that tree (including repeats)
                        leafIndices = trees[treeIdx].apply(bootData) # Identify which leaf they end up in
                        for idx,leafIdx in enumerate(leafIndices): # For each point in the bootstrap sample...
                            trainObsInLeaves[(treeIdx,leafIdx)].append(bootIndices[idx]) # add its index to the leaf it ends up in in the tree

                    # Calculate l
                    lDict[(t,xTrain.shape[0],k)].append(np.mean([tree.get_n_leaves() for tree in forest.estimators_]))

                    # Calculate the recurrence for leaf-weighted version (non-bagged trees)
                    sumsOfSquaredTotalObs = []
                    for i in range(int(t*T)):
                        relevantKeys = [(i1,i2) for i1,i2 in trainObsInLeaves.keys() if i1 == i]
                        acc = 0
                        for key in relevantKeys:
                            totalElsInLeaf = len(trainObsInLeaves[key])
                            acc += totalElsInLeaf*totalElsInLeaf
                        sumsOfSquaredTotalObs.append(acc)

                    powElement = np.prod([1 - summation/((3*T/4)*(3*T/4)) for summation in sumsOfSquaredTotalObs])
                    numerator = -(3*T/4)*(3*T/4)*powElement + (3*T/4)*(3*T/4)
                    denominator = np.sum(sumsOfSquaredTotalObs)

                    # Calculate the recurrence for leaf-weighted version (bagged trees)
                    # sumsOfSquaredUniqueObs = []
                    # sumsOfSquaredTotalObs = []
                    # for i in range(int(t*T)):
                    #     relevantKeys = [(i1,i2) for i1,i2 in trainObsInLeaves.keys() if i1 == i]
                    #     denominatorAcc = 0
                    #     numeratorAcc = 0
                    #     for key in relevantKeys:
                    #         totalElsInLeaf = len(trainObsInLeaves[key])
                    #         denominatorAcc += totalElsInLeaf*totalElsInLeaf
                    #         uniqueEls = np.unique(trainObsInLeaves[key]).shape[0]
                    #         numeratorAcc += uniqueEls*uniqueEls
                    #     sumsOfSquaredUniqueObs.append(numeratorAcc)
                    #     sumsOfSquaredTotalObs.append(denominatorAcc)
                    # powElement = np.prod([1 - summation/((3*T/4)*(3*T/4)*c) for summation in sumsOfSquaredUniqueObs])
                    # numerator = -(3*T/4)*(3*T/4)*powElement + (3*T/4)*(3*T/4)
                    # denominator = np.sum(sumsOfSquaredTotalObs)
                    
                    recDict[(t,xTrain.shape[0],k)].append(numerator/denominator)

                    
                    
                    # For each data set, I now go through with my usual permutation hypothesis testing approach but now try the dynamic programming approach. Rather than 
                    # computing actual distances, I'm just going to tally the number of distances I would have had to compute
                    overallComputations = 0
                    uniqueComputations = 0
                    
                    xPerm = deepcopy(xTest)
                    for col in range(xPerm.shape[1]): # For each column...
                        np.random.shuffle(xPerm[:,col]) # shuffle the column
                        leafIdxMatrix = forest.apply(xPerm)
                        
                        # our dynamic programming matrix; rows are test indices, columns training indices
                        dpMatrix = np.zeros(shape = (xPerm.shape[0],xTrain.shape[0]))

                        testObsInLeaves = defaultdict(list)
                        for testIdx,obs in enumerate(leafIdxMatrix): # for each test point...
                            for treeIdx,leafIdx in enumerate(obs): # for each leaf that test points ends up in...
                                testObsInLeaves[(treeIdx,leafIdx)].append(testIdx) # accumulate

                        # Get the indices of the leaves that test points end up in for each tree
                        leafIdxDict = defaultdict(list)
                        for i in range(leafIdxMatrix.shape[1]): # for each tree...
                            # get the unique leaves reached by the test points
                            leafIdxDict[str(i)] = np.unique(leafIdxMatrix[:,i])
                        
                        for treeIdx, leafIndices in leafIdxDict.items():
                            for leafIdx in leafIndices:
                                trainObsIndices = trainObsInLeaves[(int(treeIdx),leafIdx)]
                                testObsIndices = testObsInLeaves[(int(treeIdx),leafIdx)]    

                                for trainIdx in trainObsIndices:
                                    for testIdx in testObsIndices:
                                        # If you haven't computed the distance yet, mark that it was computed and count towards both unique and overall
                                        if dpMatrix[testIdx,trainIdx] == 0:
                                            overallComputations += 1
                                            uniqueComputations += 1
                                            dpMatrix[testIdx,trainIdx] = 1
                                        # otherwise just increment overall computations
                                        else:
                                            overallComputations += 1
                        xPerm[:,col] = xTest[:,col] # Change our permuted column back so that it doesn't mess up future iterations
                    overallComputationsDict[(t,xTrain.shape[0],k)].append(overallComputations/xTest.shape[0]/xTest.shape[1]) # average computations per test observation, per feature
                    uniqueComputationsDict[(t,xTrain.shape[0],k)].append(uniqueComputations/xTest.shape[0]/xTest.shape[1]) # average computations per training observation, per feature
    dataGenDict[gen.__name__].append(overallComputationsDict)
    dataGenDict[gen.__name__].append(uniqueComputationsDict)
    dataGenDict[gen.__name__].append(lDict)
    dataGenDict[gen.__name__].append(recDict)



In [ ]:
markers = ['o','v','s','P']

for k in dataGenDict.keys():
    overall = dataGenDict[k][0]
    unique = dataGenDict[k][1]
    l = dataGenDict[k][2]
    weightedRec = dataGenDict[k][3]

    overallMeans = {k : np.mean(v) for k,v in overall.items()}
    uniqueMeans = {k : np.mean(v) for k,v in unique.items()}
    lMeans = {k : np.mean(v) for k,v in l.items()}
    weightedRecMeans = {k : np.mean(v) for k,v in weightedRec.items()}


    propUniqueSim = {k : uniqueMeans[k]/overallMeans[k] for k in overallMeans.keys()}

    #propUniqueRec = {k : (-v*(1 - (1-1/np.e)/v)**(k[0]*k[1]*4/3) + v)/(k[0]*k[1]*4/3) for k,v in lMeans.items()} # For bagging
    propUniqueRec = {k : (-v*(1 - 1/v)**(k[0]*k[1]*4/3) + v)/(k[0]*k[1]*4/3) for k,v in lMeans.items()} # For non-bagging
    fig,axs = pyplot.subplots(ncols = len(Ts), figsize = (16,4), sharey = True)
    for figIdx,T in enumerate(Ts):
        for kIdx,msl in enumerate(ks):
            instanceUniquePropSim = {k : v for k,v in propUniqueSim.items() if k[1] == T*0.75 and k[2] == msl} # 0.75 comes from the train-test split being 75-25
            instanceUniquePropRec = {k : v for k,v in propUniqueRec.items() if k[1] == T*0.75 and k[2] == msl}
            instanceUniquePropWeightedRec = {k : v for k,v in weightedRecMeans.items() if k[1] == T*0.75 and k[2] == msl}
            axs[figIdx].plot(ts,instanceUniquePropSim.values(), marker = markers[kIdx], color = 'r',markersize = 7)
            axs[figIdx].plot(ts,instanceUniquePropRec.values(), marker = markers[kIdx], color = 'b',markersize = 7)
            axs[figIdx].plot(ts,instanceUniquePropWeightedRec.values(), marker = markers[kIdx], color = 'g', markersize = 7)
            axs[figIdx].set(title = f'{int(T*0.75)} Train Obs.', ylabel = 'proportion of computations unique')
            axs[figIdx].label_outer()
    fig.supxlabel('trees (as proportion of data set size)',y = -0.05)